In [12]:
from langchain.chat_models import init_chat_model
from langchain_postgres import PGVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import tool
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

True

In [13]:
if not os.environ.get("GOOGLE_API_KEY"):
  print("Please set the GOOGLE_API_KEY environment variable.")

model = init_chat_model("google_genai:gemini-2.5-flash")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [14]:
if not os.environ.get("SUPABASE_CONNECTION_STRING"):
    print("Please set the SUPABASE_CONNECTION_STRING environment variable.")

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="lilianweng_blog",
    connection=os.environ["SUPABASE_CONNECTION_STRING"],
)

In [15]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [16]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [17]:
query = (
    "What are some challenges and limitations when building LLM based Agnets."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    last_message = event["messages"][-1]
    last_message.pretty_print()

================================ Human Message =================================

What are some challenges and limitations when building LLM based Agnets.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (10cd204e-23a4-42d4-8192-19cb19cc8ac4)
 Call ID: 10cd204e-23a4-42d4-8192-19cb19cc8ac4
  Args:
    query: challenges and limitations when building LLM based Agents
================================= Tool Message =================================
Name: retrieve_context


================================== Ai Message ==================================
Tool Calls:
  retrieve_context (415e41b9-b70c-4ec6-9462-3fac32c3d82f)
 Call ID: 415e41b9-b70c-4ec6-9462-3fac32c3d82f
  Args:
    query: LLM based Agents
================================= Tool Message =================================
Name: retrieve_context


================================== Ai Message ==================================
Tool Calls:
  retrieve_context (45b63913-d7

In [18]:
# Render the agent's last message as Markdown in the Jupyter output
from IPython.display import Markdown, display
display(Markdown(last_message.text))

I am sorry, but I could not find any information regarding the challenges and limitations when building LLM-based agents in the available context.